# Tarea 2 - Máquinas de aprendizaje
Yoel Berant - rol: 201604519-8

Diego Valderas - rol: 201673549-6

Importante: para este notebook es necesario habilitar la extensión "python markdown" (que permite referenciar variables en markdown). Esta opción es parte de un pack de extensiones, que para instalar hay que seguir los pasos presentes en la siguente página: https://github.com/ipython-contrib/jupyter_contrib_nbextensions#installation.

También en la página se indica como habilitar las extensiones. Es necesario ir a file->trust notebook.

## Sección 1: Sentiment Analysis en texto

Gracias a las redes sociales, cientos de millones de personas han podido compartir sus pensamientos e ideas al mundo. Cada segundo, terabytes de mensajes via texto circulan en la red por medio de plataformas como Facebook, Twitter, páginas de prensa, páginas de criticas, mails y un largo etcetera. 

Con una cantidad tan gigantesca de información circulando, sería util desarrollar inteligencias artificiales que puedan clasificar automáticamente cada comentario, post o mail según que clase de mensaje contienen.

En esta sección se trabajará con un dataset publicado en kaggle  [[1]](#refs), en el contexto de una  competencia  organizada  por  la  Universidad  de  Stanford  [[2]](#refs) de rotten tomatoes, una famosa página web donde se comparten críticas de películas. A partir del dataset se intentará desarrollar maquinas de aprendizaje capaces de clasificar una crítica como positiva o como negativa según las palabras que aparescan en esta.

A continuación, se importa el dataset:

In [1]:
#cargar datos
import pandas as pd
ftr = open("train_data.csv", "r",  encoding="ISO-8859-1")
rows = [line.split(" ",1) for line in ftr.readlines()]
df_train = pd.DataFrame(rows, columns=['Sentiment','Text'])
df_train['Sentiment'] = (pd.to_numeric(df_train['Sentiment'])+1)/2 # 0(-) o 1(+)

fts = open("test_data.csv", "r",  encoding="ISO-8859-1")
rows = [line.split(" ",1) for line in fts.readlines()]
df_test = pd.DataFrame(rows, columns=['Sentiment','Text'])
df_test['Sentiment'] = (pd.to_numeric(df_test['Sentiment'])+1)/2 # 0(-) o 1(+)

df_train_text = df_train.Text
df_test_text = df_test.Text
labels_train = df_train.Sentiment.values
labels_test = df_test.Sentiment.values

FileNotFoundError: [Errno 2] No such file or directory: 'train_data.csv'

### a) 
Primero que nada, vale la pena hacer cierto análisis con respecto a los conjuntos que se nos presentan.

In [ ]:
import numpy as np
print("conjunto de entrenamiento:")
print("cantidad de ejemplos:", len(labels_train))
print("número de críticas positivas:", np.sum(labels_train==1))
print("número de críticas negativas:", np.sum(labels_train==0))
print("largo promedio de textos (en carácteres):",int(np.mean(list(map(lambda x: len(x),df_train_text)))))
print("varianza del largo de textos (en carácteres):",int(np.var(list(map(lambda x: len(x),df_train_text)))))
print("largo total de textos (en carácteres):",np.sum(list(map(lambda x: len(x),df_train_text))))
print("largo promedio de textos (en número de palabras, incluyendo comas y puntos):",int(np.mean(list(map(lambda x: len(x.split()),df_train_text)))))
print("varianza del largo de textos (en número de palabras, incluyendo comas y puntos):",int(np.var(list(map(lambda x: len(x.split()),df_train_text)))))
print("largo total de textos (en número de palabras, incluyendo comas y puntos):",np.sum(list(map(lambda x: len(x.split()),df_train_text))))

print("\nconjunto de prueba:")
print("cantidad de ejemplos:", len(labels_test))
print("número de críticas positivas:", np.sum(labels_test==1))
print("número de críticas negativas:", np.sum(labels_test==0))
print("largo promedio de textos (en carácteres):",int(np.mean(list(map(lambda x: len(x),df_test_text)))))
print("varianza del largo de textos (en carácteres):",int(np.var(list(map(lambda x: len(x),df_test_text)))))
print("largo total de textos (en carácteres):",np.sum(list(map(lambda x: len(x),df_test_text))))
print("largo promedio de textos (en número de palabras, incluyendo comas y puntos):",int(np.mean(list(map(lambda x: len(x.split()),df_test_text)))))
print("varianza del largo de textos (en número de palabras, incluyendo comas y puntos):",int(np.var(list(map(lambda x: len(x.split()),df_test_text)))))
print("largo total de textos (en número de palabras, incluyendo comas y puntos):",np.sum(list(map(lambda x: len(x.split()),df_test_text))))

Observando, los datos observados en el conjunto de prueba y en el de entrenamiento son muy similares.

### b)
El siguiente paso es crear un conjunto de entrenamiento y uno de prueba (a partir del conjunto inicial de entrenamiento). Se asignará el 70% de los datos al conjunto de entrenamiento y el 30% restante al de prueba.

A su vez, parte del conjunto de entrenamiento (df_train_text) se reservará para el conjunto de validación (df_val_text). Este es el conjunto en el que se prueba el modelo durante el entrenamiento en lugar de usarlo para entrenarlo.

A parte, se definen los conjuntos de labels labels_val y labels_train, que serían los valores esperados correspondientes a cada crítica en df_val_text y df_train_text respectivamente, o los conjuntos $y$, desde un punto de vista de modelos de clasificación. Si el valor $l$ de labels_train es igual a 0, quiere decir que la crítica $l$ correspondiente a df_train_text es negativa.  Si el valor $l$ de labels_train es igual a 1, significa que la crítica $l$ de df_train_text es positiva. Lo mismo con labels_val y df_val_test.


In [ ]:
#30% de validacion
from sklearn.model_selection import train_test_split
df_train_text, df_val_text, labels_train, labels_val  =train_test_split(df_train_text, labels_train, test_size=0.3, random_state=None)
#entrenamiento (30%)
#validacion (70%)

### c)
Considerando que las críticas están en inglés, el cual es un idioma con muchas palabras, prefijos, sufijos, pronombres, preposiciones, etc., es necesario normalizar la estructura de los textos puesto que una gran cantidad de palabras posibles significaría un costo gigante de ejecución por la maldición de la dimensionalidad. Es mucho más fácil si nuestro "diccionario" tiene 1000 palabras que si tuviera 5000, por ejemplo.

Por ende, a continuación se reducirá el vocabulario de distintas formas: se pasarán los textos a minúsculas (lower casing, ejemplo: "Hello"$\rightarrow$"hello") para igualarlos, se reducirán las múltiples letras, se eliminarán artículos, pronombres, preposiciones y otras palabras que no aporten significado a la frase en cuanto al sentimiento que se busca distinguir (stop word removal [[3]](#refs)) y se ocupará la técnica de "lematización" ([[4]](#refs)), que para cada palabra busca y reemplaza por su "lemma" o palabra base, según el contexto. Por ejemplo, el lemma de "better" (mejor) es "good" (bueno), o el lemma de "meeting" puede ser "meet" (conocer) o la misma palabra meeting, puesto que puede significar "reunión" o "conociendo" según el contexto en que se mencione.

Es clave que el procesamiento reduzca las palabras redundantes y elimine a las palabras que no aporten ningún significado en el juicio de la positividad o negatividad de la crítica. Es importante también que no se cambie el sentido de las palabras, puesto que podría influir erróneamente en el juicio mencionado. Por ejemplo, si se cambia la palabra "joyless" (sin alegría) a "joy" (alegría) solo porque la segunda se podría considerar como "base" para la primera, se cambiaría completamente el sentido de la palabra, de negativo a positivo.

In [ ]:
import re, time
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize

#recursos necesarios
import nltk
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')

def base_word(word):
    wordlemmatizer = WordNetLemmatizer()
    return wordlemmatizer.lemmatize(word) 

#filtro
def word_extractor(text,printer=False):
    
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text) #substitute multiple letter by two
    words = ""
    wordtokens = [ base_word(word.lower()) for word in word_tokenize(text) ]
    for word in wordtokens:
        if word not in commonwords: #delete stopwords
            words+=" "+word
    if printer:
        print(text,"->",words)
        print("")
    return words
 #try yourself

A continuación, algunos ejemplos de frases en inglés "filtradas" por este proceso:

In [ ]:
word_extractor("I love to eat cake",True)
word_extractor("I love eating cake",True)
word_extractor("I loved eating the cake",True)
word_extractor("I do not love eating cake",True)
word_extractor("I don't love eating cake",True)

Como se ve en los ejemplos, se eliminan palabras neutras como "to" y "the". Sin embargo, hay que observar en el cuarto ejemplo, la frase cambia a un sentido totalmente opuesto de lo que se quiso decir, debido a que se eliminó la palabra "not" ("no"). Es decir, se cambia "i do not love eating cake" ("a mí no me gusta comer pastel") a "love eating cake" ("me gusta comer pastel"), lo cual es lo contrario a lo que se dice originalmente. Además, la palabra "eating" (comiendo) no es transformada a "eat" (comer), es decir, que una palabra que pudo haberse generalizado no se generalizó. Por último, en el último ejemplo quedó la palabra "n't" que no tiene mucho significado más que venir de "not".

A continuación, se aplicará este filtro a los textos con los que se trabajaran:

In [ ]:
texts_train = [word_extractor(text) for text in df_train_text]
texts_val = [word_extractor(text) for text in df_val_text]
texts_test = [word_extractor(text) for text in df_test_text]

### d)

El paso siguiente es construir una representación vectorial de los datos de entrada para ser manejados, usados y clasificados por el modelo. Los datos de entrada corresponderán a una matriz. Cada fila de la matriz corresponderá a una de las críticas y cada columna a una de las palabras del diccionario (filtrado en el inciso anterior). El valor en cada casilla corresponde al número de veces en que una palabra aparece en un texto, siendo la fila $i$ y la columna $j$, el valor de la casilla $(i,j)$ indica el número de veces que la palabra representada por $j$ aparece en la crítica $i$.

Desde el punto de vista de un modelo de clasificación, las filas representarían a las instancias ($x^{l}$) y las columnas al vector de pesos ($\overrightarrow{w}$). Se busca entonces fijar el peso de cada palabra.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 1), binary=False) #TF representation
vectorizer.fit(texts_train)
features_train = vectorizer.transform(texts_train)
features_test = vectorizer.transform(texts_test)
features_val = vectorizer.transform(texts_val)

vocab = vectorizer.get_feature_names()#se arma el vocabulario


dist=list(np.array(features_train.sum(axis=0)).reshape(-1,))

También, en cada conjunto de entrada (entrenamiento, validación y prueba) se ordenarán las columnas según la cantidad total de veces que aparece cada palabra en los textos. A continuación, se mostrarán las 20 palabras más repetidas en cada conjunto de entrada:

In [ ]:
#sortear palabras por frequencia
#train:
sum_words_train = features_train.sum(axis=0) 
words_freq_train = [(word, sum_words_train[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq_train =sorted(words_freq_train, key = lambda x: x[1], reverse=True)
#val:
sum_words_val = features_val.sum(axis=0) 
words_freq_val = [(word, sum_words_val[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq_val =sorted(words_freq_val, key = lambda x: x[1], reverse=True)
#test:
sum_words_test = features_test.sum(axis=0) 
words_freq_test = [(word, sum_words_test[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq_test =sorted(words_freq_test, key = lambda x: x[1], reverse=True)

N=20

print("Las ",N," palabras más repetidas en train:")
for word, freq in words_freq_train[:N-1]:
    print(word,",",freq,"veces")

print("\nLas ",N," palabras más repetidas en val:")
for word, freq in words_freq_val[:N-1]:
    print(word,",",freq,"veces")
    
print("\nLas ",N," palabras más repetidas en test:")
for word, freq in words_freq_test[:N-1]:
    print(word,",",freq,"veces")

#2487 frases en test, 7952 palabras posibles
#cada fila de la matriz representa a una frase y cada columna a una palabra del diccionario. Los valores son la frequencia de la palabra en la frase

### e)
Para tener una visualización distinta de los textos, se hará a continuación una reducción de dimensionalidad al conjunto de entrenamiento mediante LSA (Latent Semantic Analysis) [[5]](#refs). En pocas palabras, en lugar de que cada texto se represente con una fila de largo igual a 7957 (el número total de palabras en el vocabulario), se representará como una fila de largo igual a 2, donde cada uno de los dos valores representaría que tanto se usa una de las dos "metapalabras".

In [ ]:

import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD #LSA
import matplotlib.patches as mpatches

model = TruncatedSVD(n_components=2)
model.fit(features_train)

x_plot = model.transform(features_train)
plt.figure(figsize=(10,5))


colours=['purple','yellow']

plt.scatter(x_plot[:,0], x_plot[:,1],c=labels_train)
pop_a = mpatches.Patch(color='purple', label='críticas negativas')
pop_b = mpatches.Patch(color='yellow', label='críticas positivas')

plt.title("críticas de entrenamiento, dimensionalmente reducidas a dos metapalabras")

plt.legend(handles=[pop_a,pop_b])
plt.grid=(True)
plt.xlabel("presencia de metapalabra 1")
plt.ylabel("presencia de metapalabra 2")
plt.show()

Como se puede ver en el gráfico de arriba, distintas frases se van agrupando según la naturaleza de las palabras o en este caso, el valor de las dos metapalabras.
Cabe destacar que el sentimiento de cada frase no impera en cada grupo que se formó. Es decir, hay frases tanto positivas como negativas agrupadas en mismos sectores. Esto último significa que dos frases con palabras similares pueden ser de distintos sentimientos, lo que quiere decir que, por lo menos en esta dimensionalidad es difícil diferenciar palabras positivas y negativas según el valor de las metapalabras.

### f)
A continuación, se comenzará a entrenar distintos modelos de clasificación. El primero será un modelo de regresión logística con penalizador norma $l_{2}$. Este modelo restringe los valores máximos y mínimos de los pesos de las palabras, para que ninguna palabra pese mucho más (en magnitud) que las demás.

In [ ]:
from sklearn.linear_model import LogisticRegression
from ipywidgets import interact,interactive, fixed

def do_LOGIT(x,y,xv,yv, param):
    #print("Param C= ",param)
    model= LogisticRegression()
    model.set_params(C=param)
    model.fit(x,y)
    train_acc = model.score(x,y)
    test_acc = model.score(xv,yv)
    #print("logit: ","train acc: ",train_acc, "test (val) acc: ",test_acc," param: ",param)
    return model, train_acc, test_acc

Cs = [10**np.int(i) for i in np.arange(-4,4)]
train_accs_log=[]
test_accs_log=[]

for c in Cs:
    model, train_acc, test_acc = do_LOGIT(features_train,labels_train,features_val,labels_val, param=c)
    train_accs_log.append(train_acc)
    test_accs_log.append(test_acc)



En el siguiente gráfico, se muestran los niveles de exactitud de predicción obtenidos tanto en el conjunto de entrenamiento como en el de evaluación según el parámetro C, el cual indica la norma a la cual se restringen los pesos (entre menor es C, mayor es la regularización).

In [ ]:
plt.figure(figsize=(7,4))
plt.plot(Cs, train_accs_log, label="entrenamiento")
plt.plot(Cs, test_accs_log, label="evaluación")
plt.xscale("log")
plt.xlabel("valor de C")
plt.ylabel("exactitud")
plt.title("nivel de exactitud con regresión logística norma l2 según parámetro c")
#plt.xlim(left=0.1,right=10)
#plt.ylim(bottom=0.7,top=0.704)
plt.legend()
plt.show()

#tldr: un C grande provoca overfitting
#en C=1 se alcanza lo optimo

El mayor valor de exactitud de esta regresión ocurre cuando $C=1$, alcanzando un nivel de exactitud de {{test_accs_log[3]}}.

### g)

Ahora se usará un modelo de máquina de soporte vectorial (SVM). Este modelo consiste en, ajustando los pesos de las palabras, encontrar al hiperplano que pueda separar mejor a las críticas negativas con las positivas además de regularizar los pesos de un modo similar al que regularizaron los modelos de regresión logística del inciso anterior. Esto se logra resolviendo el problema de optimización:

$$
min_{w,b} \frac{1}{2}\mid\mid w\mid\mid^{2}
\\s.t: y^{(l)}(w^{T}x^{(l)}+b)\geq 1,\space\forall l
$$

Donde $w$ es el vector de pesos de las palabras, $b$ es el vector bias, $y^{(l)}$ es la clasificación del texto o crítica $l$ (en este caso -1 si es negativa y 1 si es positiva) y $x_{(l)}$ representa a la crítica $l$, como se describió en el inciso d).

Resolver este problema equivale a resolver el problema "dual", el cual consiste en encontrar el vector $\alpha$ tal que:

$$
max_{a} \sum_{l}{a_{l}}-\frac{1}{2}\sum_{l,m}{a_{l}a_{m}y^{(l)}y^{(m)}<x_{l},x_{m}>}
\\s.t: \sum_{l}{a_{l}y^{(l)}}=0, \forall l
$$

Donde $<x_{l},x_{m}>$ denota $x_{l}^{T}x_{m}$

Una vez resuelto este problema, para clasificar un nuevo texto $x$ (el cual está vectorizado como cualquier vector $x_{l}$), se ocupa la función:

$$
f(x)=sign(\sum_{l}{w^{T}x+b})=sign(\sum_{l}{a_{l}y{(l)}<x_{l},x_{m}>+b})
$$

Este método utiliza distintos "kernels", que son fórmulas para cambiar el espacio en que habitan los datos y así poder separar los datos en el espacio no-linealmente. Así, si reemplazamos $<x_{l},x_{m}>$ por $k(x_{l},x_{m})$ (la función kernel), donde k es la función kernel el problema de optimización se transforma:

$$
max_{a} \sum_{l}{a_{l}}-\frac{1}{2}\sum_{l,m}{a_{l}a_{m}y^{(l)}y^{(m)}k(x_{l},x_{m})}
\\s.t: \sum_{l}{a_{l}y^{(l)}}=0, \forall l
$$

y la función de clasificación:

$$
f(x)=sign(\sum_{l}{w^{T}\phi(x)+b})=sign(\sum_{l}{a_{l}y{(l)}k(x_{l},x_{m})+b})
$$

Donde $\phi(x)$ es una función relacionada a la función kernel.

Se probarán algunos modelos con distintos tipos de kernels y varios parámetros de regularización. Los kernels son:
* lineal: $k(x_{l},x_{m})=x_{l}^{T}x_{m}$ (acá no se hace una transformación)
* polinomial: $k(x_{l},x_{m})=(\gamma x_{l}^{T}x_{m} + c_{0})^{p}$ (donde $\gamma, c_{0}$ y $p$ son parámetros escalares)
* RBF o gausseano: $k(x_{l},x_{m})=exp(-\gamma\mid\mid x_{l}^{T}x_{m}\mid\mid^{2})$
* sigmoidal:  $k(x_{l},x_{m})=\tanh(\gamma x_{l}^{T}x_{m} + c_{0})$

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Kernel_Machine.svg/1200px-Kernel_Machine.svg.png" title="smv" width="40%" />

In [ ]:
from sklearn.svm import SVC as SVM #SVC is for classification
def do_SVM(x,y,xv,yv, param, kernel='linear',prob=False):
    C=param
    #print("Param C= ",C, 'Kernel= ', kernel)
    model= SVM(probability=prob)
    if kernel=="lineal":
        model.set_params(C=C,kernel=kernel) #try rbf and linear at least
    else:
        model.set_params(C=C,kernel=kernel,gamma="scale",degree=2)
        
    model.fit(x,y)
    train_acc = model.score(x,y)
    test_acc = model.score(xv,yv)
    return model, train_acc, test_acc
Cs = [10**np.int(i) for i in np.arange(-4,4)]

train_accs_svm_linear=[]
test_accs_svm_linear=[]

print("entrenando y probando modelos de kernel lineal...")
for c in Cs:
    model, train_acc, test_acc = do_SVM(features_train,labels_train,features_val,labels_val, param=c, kernel="linear")
    train_accs_svm_linear.append(train_acc)
    test_accs_svm_linear.append(test_acc)

train_accs_svm_rbf=[]
test_accs_svm_rbf=[]

print("entrenando y probando modelos de kernel rbf...")
for c in Cs:
    model, train_acc, test_acc = do_SVM(features_train,labels_train,features_val,labels_val, param=c, kernel="rbf")
    train_accs_svm_rbf.append(train_acc)
    test_accs_svm_rbf.append(test_acc)
    
train_accs_svm_poly=[]
test_accs_svm_poly=[]

print("entrenando y probando modelos de kernel polinomial...")
for c in Cs:
    model, train_acc, test_acc = do_SVM(features_train,labels_train,features_val,labels_val, param=c, kernel="poly")
    train_accs_svm_poly.append(train_acc)
    test_accs_svm_poly.append(test_acc)

train_accs_svm_sigmoid=[]
test_accs_svm_sigmoid=[]

print("entrenando y probando modelos de kernel sigmoide...")
for c in Cs:
    model, train_acc, test_acc = do_SVM(features_train,labels_train,features_val,labels_val, param=c, kernel="sigmoid")
    train_accs_svm_sigmoid.append(train_acc)
    test_accs_svm_sigmoid.append(test_acc)

print("modelos entrenados!")

En el siguiente gráfico, se muestran los niveles de exactitud de predicción obtenidos tanto en el conjunto de entrenamiento como en el de evaluación según el parámetro C, el cual indica la norma a la cual se restringen los pesos y según el tipo de kernel seleccionado.

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(Cs, train_accs_svm_linear, label="kernel lineal(entrenamiento)",c="darkred")
plt.plot(Cs, test_accs_svm_linear, label="kernel lineal(evaluación)",c="red")
plt.plot(Cs, train_accs_svm_rbf, label="kernel rbf(entrenamiento)",c="blue")
plt.plot(Cs, test_accs_svm_rbf, label="kernel rbf(evaluación)",c="aqua")
plt.plot(Cs, train_accs_svm_poly, label="kernel polinomial(entrenamiento)",c="purple")
plt.plot(Cs, test_accs_svm_poly, label="kernel polinomial(evaluación)",c="magenta")
plt.plot(Cs, train_accs_svm_sigmoid, label="kernel sigmoidal(entrenamiento)",c="green")
plt.plot(Cs, test_accs_svm_sigmoid, label="kernel sigmoidal(evaluación)",c="lime")
plt.xscale("log")
plt.xlabel("valor de C")
plt.ylabel("exactitud")
plt.title("nivel de exactitud con SVM según parámetro de regresión C y según kernel")
#plt.xlim(left=0,right=100)
plt.legend()
plt.show()
#printlist(zip(test_accs_svm_linear,Cs))
#printlist(zip(test_accs_svm_rbf,Cs))
#printlist(zip(test_accs_svm_poly,Cs))
#printlist(zip(test_accs_svm_sigmoid,Cs))

Considerando que nuevamente se usaron como valores de C potencias de 10 (este C corresponde al grado de regularización, no al costo de los epsilons), si hay que elegir un kernel y valor de C por mayor exactitud de evaluación y menor diferencia entre exactitud de entrenamiento y evaluación (menor overfitting), el modelo que se escoge es el de kernel rbf con $C=100$. La exactitud de evaluación en este punto es de {{test_accs_svm_rbf[5]}}.


### h)

Se utilizarán a continuación modelos "k-NN" (k nearest neighbors). Estos modelos consisten en, dado un valor k clasificar un punto al tomar los k puntos más cercanos a este y verificar cual clasificación se repite más. 

<img src="https://miro.medium.com/max/810/0*uNbO79MrS7jvY4qp.png" title="knn" width="20%" />

Se probarán distintos modelos con distintos valores de k.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
def do_KNN(x,y,xv,yv, param):
    model = KNeighborsClassifier()
    #print("Param K= ",param)
    model.set_params(n_neighbors=param)
    model.fit(x,y)
    train_acc = model.score(x,y)
    test_acc = model.score(xv,yv)
    return model, train_acc, test_acc

steps=50
Ks = np.arange(3, features_train.shape[0], steps)

train_accs_knn=[]
test_accs_knn=[]

#print('probando KNN...')
for k in Ks:
    model, train_acc, test_acc = do_KNN(features_train,labels_train,features_val,labels_val, param=k)
    train_accs_knn.append(train_acc)
    test_accs_knn.append(test_acc)
#print('KNN probado!')

En el siguiente gráfico, se muestran los niveles de exactitud de predicción obtenidos tanto en el conjunto de entrenamiento como en el de evaluación según el parámetro k.

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(Ks, train_accs_knn, label="entrenamiento")
plt.plot(Ks, test_accs_knn, label="evaluación")
plt.xlabel("valor de k")
plt.ylabel("exactitud")
plt.title("nivel de exactitud con kNN según parámetro k")
plt.legend()
plt.show()
#optimo elegido, k=300
#print(list(zip(test_accs_knn,Ks)))

El mayor valor de exactitud de prueba registrado en este caso es de {{max(test_accs_knn)}}. Este método no parece ser muy superior a los que hemos visto.

### i)
Lo siguiente es entrenar árboles de decisiones. Estos "arboles" generados son secuencias que buscan clasificar una entrada según "preguntas" relacionadas a una valoriable individualmente. El objetivo es encontrar los valores de variables que mejor separan a las críticas porsitivas de las negativas, ir preguntando en secuencia si las variables de la instancia de entrada son mayores o menores a los valores encontrados, e ir separando según eso. 

<img src="https://mobilemonitoringsolutions.com/wp-content/uploads/2019/01/931260162" title="tree" width="30%" />


Por ejemplo, si se obtiene que la presencia de la palabra "bad"(malo) dos o más veces en una crítica es lo que mejor la distingue como una crítica negativa (porque una crítica negativa muy probablemente tendrá esa palabra), una crítica que contenga esta palabra por lo menos 2 veces (o las veces que defina el modelo) probáblemente será catalogada instantáneamente como negativa. Si no contiene esa palabra, se pregunta entonces por la segunda palabra que particione más y así sucesivamente.

La creación de un árbol de clasificación depende de dos parámetros: la altura máxima, es decir, el número máximo de "preguntas" hasta llegar a una conclusión final y el grado mínimo de división de entrenamiento, es decir, la cantidad mínima de ejemplos del conjunto de entrenamiento que terminen en una partición luego de una pregunta durante el entrenamiento.

A continuación, se entrenarán modelos de árboles de clasificación modificando estos dos parámetros:

In [ ]:
from sklearn.tree import DecisionTreeClassifier as Tree
def do_Tree(x,y,xv,yv, param_d=None, param_m=2):
    model= Tree()
    #print("Param Max-D= ",param_d, 'Min-samples-S= ', param_m)
    model.set_params(max_depth=param_d, min_samples_split=param_m) 
    model.fit(x,y)
    train_acc = model.score(x,y)
    test_acc = model.score(xv,yv)
    return model, train_acc, test_acc

D_steps=500
S_steps=350
Depths = np.arange(1, features_train.shape[1], D_steps, dtype=np.float64) #choose steps
SamplesS = np.arange(2, features_train.shape[0] , S_steps, dtype=np.int_ ) #choose steps
train_accs_dt=np.zeros((len(SamplesS),len(Depths)),dtype=np.float64)
test_accs_dt=np.zeros((len(SamplesS),len(Depths)),dtype=np.float64)
i=0
j=0

print('probando arboles de desicion...')
for s in SamplesS:
    for d in Depths:        
        model, train_acc, test_acc = do_Tree(features_train,labels_train,features_val,labels_val, param_d=d, param_m=s)
        train_accs_dt[i][j]=train_acc
        test_accs_dt[i][j]=test_acc
        j+=1
    j=0
    i+=1
print('arboles probados!')

A continuación, se expone dos gráficos, uno indicando exactitud de entrenamiento y el otro de evaluación. Debido a que esta vez son dos parámetros que varían, ¡los gráficos son tridimensionales!

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS9R3HqeS_bRck46VZZU8F9q9DVGq4XPbStP_YiJsaMnDlThxri" title="dog" width="20%" />


In [ ]:
from mpl_toolkits.mplot3d import Axes3D


X, Y = np.meshgrid(Depths, SamplesS)
Z1 = train_accs_dt
Z2 = test_accs_dt

fig1 = plt.figure(figsize=(10,7))
ax1 = fig1.gca(projection='3d')
ax1.set_title("exactitud lograda en entrenamiento")
ax1.set_xlabel("profundidad máxima")
ax1.set_ylabel("cantidad mínima de splits en entrenamiento")
ax1.set_zlabel("exactitud lograda")
surf1 = ax1.plot_surface(X, Y, Z1, rstride=1, cstride=1, cmap='hot', linewidth=0, antialiased=True)


fig1.colorbar(surf1, shrink=0.5, aspect=5)
ax1.view_init(azim=20, elev=60)
ax1.set_zlim(Z1.min()-0.1, Z1.max()+0.1)

fig2=plt.figure(figsize=(10,7))
ax2 = fig2.gca(projection='3d')
ax2.set_title("exactitud lograda en evaluación")
ax2.set_xlabel("profundidad máxima")
ax2.set_ylabel("cantidad mínima de splits en entrenamiento")
ax2.set_zlabel("exactitud lograda")
surf2 = ax2.plot_surface(X, Y, Z2, rstride=1, cstride=1, cmap='Blues_r', linewidth=0, antialiased=True)
fig2.colorbar(surf2, shrink=0.5, aspect=5)
ax2.view_init(azim=20, elev=60)
ax2.set_zlim(Z2.min()-0.1, Z2.max()+0.1)
# optimo elegido, cantidad minima=250, profundidad máxima=3750


Analizando los gráficos, nos percatamos de que cuando la cantidad mínima de splits en entrenamiento es muy grande, la eficiencia baja. Así mismo, la profundidad máxima no es un factor muy influyente en el error.

Por alcanzar un mayor nivel de exactitud de evaluación (lo cual se traduce como menor overfitting), el modelo optimo escogido es de profundidad máxima igual a 3750 y grado mínimo de división de entrenamiento igual a 250, que alcanza una exactitud de evaluación de 0.62.

### j)

A continuación, se desarrollarán modelos de redes neuronales artificiales o "ANN's". Estos modelos funcionan de manera similar a las regresiones logísticas, con la diferencia de que hay más vectores pesos organizados por capas. Cada peso de cada capa es una combinación lineal de los pesos de la capa anterior, comenzando por la capa de entrada, que serían los pesos de las entradas. La siguiente imagen ilustra un ejemplo simplificado:


<img src="https://icdn6.digitaltrends.com/image/digitaltrends/artificial_neural_network_1-791x388.jpg" title="ann" width="50%" />

A cada nodo perteneciente a una capa se le llama "neurona", y tiene un peso que se actualiza durante el entrenamiento.

Los modelos con los que los que se trabajarán consistirán en la capa de entrada, una capa oculta con un número de neuronas $N_{h}$ que es potencia de 2 (el cual se irá variando) y la capa de salida, que debería indicar luego del entrenamiento la probabilidad de que el texto de entrada sea positivo.

Estos modelos tendrán un tamaño de batch de 128. El batch es el conjunto máximo de datos de entrenamiento usados para calcular errores y así actualizar los pesos de las neuronas durante el entrenamiento. Se entrenará el modelo con 25 iteraciones al dataset (o epochs). Los modelos se entrenan con SGD, con tasa de aprendizaje 0.1; en otras palabras, durante el entrenamiento, los pesos de las neuronas se irán actualizando a una razón de 0.1 en relación a los errores.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

'''
si los imports de arriba no funcionan, probar:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
'''

def do_ANN(x,y, xv,yv, param):
    #print("Neuron hidden = ",param)
    model = Sequential()
    model.add(Dense(units=param, input_dim=x.shape[1], activation="sigmoid"))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer=SGD(lr=0.1), loss="binary_crossentropy", metrics=["accuracy"])
    model.fit(x, y, epochs=25, batch_size=128, verbose=0)
    train_acc = model.evaluate(x,y, verbose=0)[1] #in position 0 is the loss
    test_acc = model.evaluate(xv,yv, verbose=0)[1]
    return model, train_acc, test_acc
N_h = [2**i for i in range(1,8)]
#N_h = [2**i for i in range(1,12)]

train_accs_ann=[]
test_accs_ann=[]

print('probando redes neuronales (se pide paciencia)...')
for n in N_h:
    #print(n)
    model, train_acc, test_acc = do_ANN(features_train,labels_train,features_val,labels_val, param=n)
    #print("train acc: ", train_acc,"test_acc: ",test_acc)
    train_accs_ann.append(train_acc)
    test_accs_ann.append(test_acc)
print('redes neuronales probadas!')

In [ ]:

plt.figure(figsize=(10,6))
plt.plot(N_h, train_accs_ann, label="entrenamiento")
plt.plot(N_h, test_accs_ann, label="evaluación")
#plt.xlim(left=0,right=150)
plt.xscale('log')
plt.xlabel("valor de Nh")
plt.ylabel("exactitud")
plt.legend()
plt.show()
#optimo elegido: Nh=16
#print(list(zip(test_accs_ann,N_h)))

La máximo exactitud de prueba registrada es de {{max(test_accs_ann)}}.

Nota: Si se repite varias veces el proceso de entrenar redes neuronales con distintos valores de n, es facil percatarse de que los resultados difieren drásticamente y los gráficos no se parecen en nada. Es por eso que no decidí hacer algún otro comentario (pasó lo mismo con knn). 

### h)
Si consideramos como medida de desempeño a la exactitud de evaluación, el mejor modelo obtenido en los incisos anteriores es el de svm de kernel rbf con valor de regresión C=100, que alcanza un nivel de desempeño de {{test_accs_svm_rbf[5]}}.

Esto quiere decir que un 70% (aprox) de las críticas fueron clasificadas exitosamente durante la evaluación de este modelo. Considerando el potencial de ambigüedad que lleva con sigo el lenguaje humano y, más específicamente, el idioma inglés, 0.71 es un nivel de desempeño alto, considerando que el resto de los modelos trabajados difícilmente pudieron superar los 0.65. Aun así, sigue existiendo un 30% (más o menos) de error que no es despreciable. Si se quiere llegar a un nivel de predicción más confiable, un valor mínimo de referencia de desempeño es 0.85, aunque nuevamente, por culpa de la ambigüedad de los idiomas, es difícil juzgar a una oración solo por sus palabras y no por el modo en que se relacionan entre sí dado un contexto dado. En otras palabras, este es un problema de clasificación que es esencialmente difícil.

### l)
A continuación, se hará uso del modelo VADER (Valence Aware Dictionary and sEntiment Reasoner) [[6]]. Este no es un modelo de máquinas de aprendizaje, sino que fue construido manualmente y, por lo tanto, no requiere de entrenamiento. El modelo entrega, dada una frase, un score de predicción léxico en cuanto a la positividad y la negatividad de la palabra. Es decir, la clasifica como positiva, negativa o neutral.

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
def vader_predict(sentences): 
    sid_obj = SentimentIntensityAnalyzer() 
    sent_v = []
    for text in sentences:
        sentiment_dict = sid_obj.polarity_scores(text) 
        if sentiment_dict["pos"] > sentiment_dict["neg"]: #based on scores
            sent_v.append(1)
        else:
            sent_v.append(0)
    return np.asarray(sent_v)

vader_pred_test = vader_predict(df_test_text) 
from sklearn.metrics import accuracy_score
accuracy_score(labels_test, vader_pred_test)

El modelo VADER entregó una predicción de los datos de evaluación (los cuales esta vez no hubo que preprocesarlos) con una eficiencia de 0.636, lo cual es menor a la eficiencia de svm con kernel rbf y C=100.

### m)

A continuación, se mostrarán las 20 palabras más positivas y las 20 más negativas según el modelo svm de kernel rbf y C=100 junto con la predicción de VADER de estas. Se considera a la positividad/negatividad de un texto se mide según la probabilidad calculadas por el modelo de ser positivo.

In [ ]:
#modelo escogido:

model_rbf,tr,te = do_SVM(features_train,labels_train,features_val,labels_val, param=100, kernel="rbf",prob=True)
print("rbf: ","train acc: ",tr, "test (val) acc: ",te)

V = len(vocab)

score_list_rbf=[]

print("calculando probabilidades para cada palabra en modelo de regresión logística...")

sid_obj = SentimentIntensityAnalyzer() 

for i in range(V):
    x_word = np.zeros((1, V))
    x_word[:,i] = 1 # only the "i" word appeared
    score_list_rbf.append((model_rbf.predict_proba(x_word)[0],vocab[i]))
print("probabilidades calculadas")

In [ ]:
from __future__ import print_function


def vader_predict_word(word, sid_obj):
    
    sentiment_dict = sid_obj.polarity_scores(word)
    if sentiment_dict["neu"]==1.0:
        return "neutral"
    elif sentiment_dict["pos"] > sentiment_dict["neg"]:
        return "positiva"
    else:
        return "negativa"
    

#se ordenan las palabras por probabilidad de positivismo
score_list_rbf.sort(key=lambda x:((x[0][0])))


sid_obj = SentimentIntensityAnalyzer() 

print('las 20 palabras mejores puntuadas según svm con rbf:')

for i in range(20):
    print(i+1,": ",score_list_rbf[i][1],", percibida por VADER como ",vader_predict_word(score_list_rbf[i][1],sid_obj))


#print([(score_list_rbf[::-1])[i][1]+"\n" for i in range(20)])
print('\nlas 20 palabras peores puntuadas según svm con rbf:')
for i in range(20):
    print(i+1,":",score_list_rbf[-i-1][1],", percibida por VADER como ",vader_predict_word(score_list_rbf[-i-1][1],sid_obj))



Si bien muchas palabras neutrales (según VADER) se "colan" a ambas listas, viéndolas en el contexto de una crítica de cine tiene sentido que hayan sido catalogadas como positivas o negativas como la regresión logística. Por ejemplo "*heart*" (*corazón*) y "*refreshing*" (refrescarse) son palabras neutrales según VADER que se podrían considerar como tal en un sentido literal, pero al elogiar una buena película, puedo decir que un actor entrega una buena actuación con el corazón o que las ideas que propone el largometraje son *"refrescantes"*. 

### n)
Para mejorar los resultados, esta vez, en lugar de usar lematización (como en el inciso c)), se hará el preprocesamiento del vocabulario con una técnica llamada stemming [[7]](#refs). Stemming, en lugar de buscar la base de la palabra según el contexto de la horación y según el significado, reduce la palabra a su base borrando prefijos y sufijos. Por ejemplo "wonderful"(maravilloso) y "fishing"(pescando) serán reducidas a "wonder"(maravilla) y "fish"(pez) respectivamente, no por lo que significan, sino porque terminan respectivamente con "ful" y "ing", que son sufijos de palabras las cuales se filtran.

Como fue mencionado en el inciso c), es necesario que la reducción de una palabra no afecte su significado, por ejemplo, la palabra "joyless"(sin alegría), a pesar de que su base es "joy"(alegría), reducirla a esta cambiaría su significado, pues necesita del sufijo "less" para indicar una negación. Por lo tanto, cambiar "joyless" a "joy" cambiaría totalmente el significado de la palabra.

In [ ]:
from nltk.stem import PorterStemmer#Más ligero y practico. Usaremos este.
#from nltk.stem import LancasterStemmer #mas lento e iterativo

porter = PorterStemmer()
#lancaster=LancasterStemmer()
print(porter.stem("joyful"))
print(porter.stem("joyless"))#NOTA: por suerte, estos "stematizadores" no modifican palabras con el prefijo "less". Es decir, la palabra no podría obtener un sentido contrario a lo que significa.

In [ ]:
'''
simmilar al inciso c)...
'''

def base_word_stm(word):
    porter = PorterStemmer()
    return porter.stem(word)

def word_extractor_stm(text,printer=False):
    
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text) #substitute multiple letter by two
    words = ""
    wordtokens = [ base_word_stm(word.lower()) for word in word_tokenize(text) ]
    for word in wordtokens:
        if word not in commonwords: #delete stopwords
            words+=" "+word
    if printer:
        print(text,"->",words)
        print("")
    return words

#ejemplos
word_extractor_stm("I love to eat cake",True)
word_extractor_stm("I love eating cake",True)
word_extractor_stm("I loved eating the cake",True)
word_extractor_stm("I do not love eating cake",True)
word_extractor_stm("I don't love eating cake",True)
 #try yourself
texts_train_stm = [word_extractor_stm(text) for text in df_train_text]
texts_val_stm = [word_extractor_stm(text) for text in df_val_text]
texts_test_stm = [word_extractor_stm(text) for text in df_test_text]

Al igual que en el inciso c), se eliminaron las preposiciones y palabras que no ocupaban (aparentemente) mucha relevancia, como *"I"*, *"the"* y *"not"*. Esta última palabra, al ser borrada, cambia el sentido de la cuarta frase tal como pasó en el inciso c), pero esto no ocurrió debido a la lematización o la stematización sino que fue filtrada por el proceso anterior.

Dicho esto, una diferencia con la lematización es que la stematización cambió la palabra "loved" a "love", puesto que detectó el sufijo "ed".

Ahora, se verificará si hay mejoras en la predicción luego de usar stemming repitiendo el proceso echo en el inciso m), usando la misma regresión logística.

In [ ]:
vectorizer_stm = CountVectorizer(ngram_range=(1, 1), binary=False) #TF representation
vectorizer_stm.fit(texts_train_stm)
features_train_stm = vectorizer_stm.transform(texts_train_stm)
features_test_stm = vectorizer_stm.transform(texts_test_stm)
features_val_stm = vectorizer_stm.transform(texts_val_stm)



model_logit,tr_lm,te_lm = do_SVM(features_train,labels_train,features_val,labels_val, param=100, kernel="rbf",prob=True)
print("svm con lematizacion: ","train acc: ",tr_lm, "test (val) acc: ",te_lm)

model_logit_stm,tr_stm,te_stm = do_SVM(features_train_stm,labels_train,features_val_stm,labels_val, param=100, kernel="rbf",prob=True)
print("svm con stemming: ","train acc: ",tr_stm, "test (val) acc: ",te_stm)



Dado que se prioriza la precisión de evaluación, se concluye que es mejor la regresión logística con lematización.

### o)
A continuación, se realizará una modificación al filtro hecho en el inciso c) (lematización). Considerando que el vocabulario resultante inicialmente consistía en más de 7000 palabras, esta vez solo se usarán las 4000 palabras más usadas. Además, no se realizará la eliminación de multiples letras.

La modificación mencionada, junto con todo el proceso de vectorización y la eliminación de las stopwords, se puede realizar en una sola línea usando la clase *TfidfVectorizer* de la librería *sklearn.feature_extraction.text*  [[8]](#refs).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#stopneg=[w for w in stopwords.words('english') if w not in ("against","don't","not","didn't","doesn't","hadn't","hasn't","haven't","isn't","mightn't","shouldn't","wasn't","weren't","won't","wouldn't")]

def lema(text,printer=False):    
    #text = re.sub(r'([a-z])\1+', r'\1\1',text) #substitute multiple letter by two
    words = ""
    #lematización
    wordtokens = [ base_word(word.lower()) for word in word_tokenize(text) ]
    for word in wordtokens:
        words+=" "+word
    if printer:
        print(text,"->",words)
        print("")
    return words


print("lematizando...")

texts_train = [lema(text) for text in df_train_text]
texts_val = [lema(text) for text in df_val_text]
texts_test = [lema(text) for text in df_test_text]



print("vectorizando...")
tfidf_model = TfidfVectorizer(binary=False, stop_words=stopwords.words('english'), ngram_range=(1, 1), max_features=4000, norm='l2', use_idf=True, sublinear_tf=False,vocabulary=None)
tfidf_model.fit(texts_train)
tfidf_train=tfidf_model.transform(texts_train)
tfidf_test=tfidf_model.transform(texts_test)
tfidf_val=tfidf_model.transform(texts_val)
print("listo")

Repitiendo el proceso de verificación hecho en m)...

In [ ]:
#de nuevo...
model_svm_tfidf,tr,te = do_SVM(tfidf_train,labels_train,tfidf_val,labels_val, param=100, kernel="rbf",prob=True)
print("reg. logística con lematizacion y tfidf: ","train acc: ",tr, "test (val) acc: ",te)

#haciendo lo mismo que en el inciso m):
vocab_tfidf = tfidf_model.get_feature_names()
Vtfidf=len(vocab_tfidf)


score_list_svm_tfidf=[]
for i in range(Vtfidf):
    x_word = np.zeros((1, Vtfidf))
    x_word[:,i] = 1 # only the "i" word appeared
    score_list_svm_tfidf.append((model_svm_tfidf.predict_proba(x_word)[0],vocab_tfidf[i]))

score_list_svm_tfidf.sort(key=lambda x:((x[0][0])))

print('\nlas 20 palabras mejores puntuadas con lematización modificada:')

for i in range(20):
    print(i+1,": ",score_list_svm_tfidf[i][1],", percibida por VADER como ",vader_predict_word(score_list_svm_tfidf[i][1],sid_obj))

#print([(score_list_logit[::-1])[i][1]+"\n" for i in range(20)])
print('\nlas 20 palabras peores puntuadas con lematización modificada:')
for i in range(20):
    print(i+1,":",score_list_svm_tfidf[-i-1][1],", percibida por VADER como ",vader_predict_word(score_list_svm_tfidf[-i-1][1],sid_obj))




La exactitud de prueba en este caso ({{te}}) es mayor a la resultante tanto con lemmatization ({{te_lm}}) sin modificaciones como con stemming ({{te_stm}}). 

Por lo tanto, nos quedaremos con este como el mejor modelo hasta ahora.

### p)
Usando la función *classification_report* de la librería *sklearn.metrics* ([[9]](#refs)), evaluaremos el mejor modelo obtenido (el del inciso anterior). Esta función entrega 4 datos por cada clase (críticas positivas o críticas negativas):

([[10]](#refs))
1. "*Precision*" de una clase: es la habilidad del clasificador de no considerar un elemento que no sea de la clase como parte de esta. Un precision de 1 quiere decir que no se consideraron elementos que no sean de la clase, mientras que un precision de 0 quiere decir que todos los elementos que se clasificaron como de la clase, en realidad no son parte de ella.
2. "*Recall*" de una clase: es la habilidad del clasificador de identificar correctamente a los elementos de la clase. Un recall de 1 quiere decir que todos los elementos que pertenecían a la clase fueron clasificados correctamente, mientras que un recall de 0 quiere decir que ninguno de los elementos de la clase se clasificó correctamente.
3. "*f1-score*" de una clase: es la media armónica entre el precision y el recall de esta.
4. *support* de una clase: es la cantidad real de elementos de la clase en la muestra (es decir, de acuerdo al label).

In [ ]:
from sklearn.metrics import classification_report

best_model=model_svm_tfidf
best_features_test=tfidf_test
best_features_train=tfidf_train


def score_the_model(model, x, y):
    print("Detailed Analysis Testing Results ...")
    print(classification_report(y, model.predict(x), target_names=['-','+']))
    
    #train_acc = model.score(x,y)
    #print("score: ",model.score(x,y))
    
score_the_model(best_model, best_features_test, labels_test)
#...

De estas mediciones, podemos concluír que al clasificador le cuesta un poco más detectar una crítica negativa que una positiva, pues el recall de "+" es mayor (lo que quiere decir que el clasificador detecta más seguido críticas positivas como positivas que negativas como negativas) y la precision de "+" es menor (lo que quiere decir que el clasificador suele detectar críticas negativas como positivas con más frecuencia que críticas positivas como negativas).

Esto nos podría dar una pista de como mejorar la eficacia del modelo, pues si variamos los pesos de las clases (dándole más importancia a la detección de una clase sobre la otra) es posible que aumente la precisión del clasificador.

### q)
A continuación, se repetirá el inciso anterior varias veces, pero variando el peso de las clases (positivos o negativos podrán pesar más menos) usando el parámetro "class weight". De esta forma, se corroborará lo dicho en el inciso anterior.

In [ ]:
print("-si los negativos pesan 10 veces más:")
b_model=best_model
classes_weights = {0: 10, 1: 1} #negativos pesan 5 veces más
b_model.set_params(class_weight=classes_weights)
b_model.fit(best_features_train, labels_train)
score_the_model(b_model, best_features_test, labels_test)
print("\n-si los negativos pesan 5 veces más:")
b_model=best_model
classes_weights = {0: 5, 1: 1} #or choose..
b_model.set_params(class_weight=classes_weights)
b_model.fit(best_features_train, labels_train)
score_the_model(b_model, best_features_test, labels_test)
print("\n-si los positivos pesan 5 veces más:")
b_model=best_model
classes_weights = {0: 1, 1: 5}
b_model.set_params(class_weight=classes_weights)
b_model.fit(best_features_train, labels_train)
score_the_model(b_model, best_features_test, labels_test)
print("\n-si los positivos pesan 10 veces más:")
b_model=best_model
classes_weights = {0: 1, 1: 10} #or choose..
b_model.set_params(class_weight=classes_weights)
b_model.fit(best_features_train, labels_train)
score_the_model(b_model, best_features_test, labels_test)

Curiosamente, los valores parecen mantenerse igual, puesto que aunque se les cambie el peso a las clases, el clasificador se encarga de automáticamente atribuirles un peso mayor o menor. 

### r)
Otra forma de probar y analizar el mejor clasificador obtenido es viendo como se comporta al tener que clasificar algunos textos, mientras tenemos acceso al contenido de esos textos. A continuación, se elegirán algunos textos aleatorios, obserbará la probabilidad calculada (predict_proba) por el clasificador de ser positiva o negativa.

In [ ]:
def clase_pred(pred):
    if(pred[0]>pred[1]):
        return "negativa"
    else:
        return "positiva"
def clase_true(true):
    if true==0.0:
        return "negativa"
    else:
        return "positiva"

test_pred = best_model.predict_proba(best_features_test)
#test_pred = log_model.predict_proba(features_test) #or ".predict"
spl = np.random.randint( 0, len(test_pred), size=15)#selección pequeña y aleatoria de ejemplos
for text, pred_s, true_s in zip(df_test_text[spl], test_pred[spl], labels_test[spl]):
    print("Clasificación verdadera: ", clase_true(true_s), "-- Probabilidades calculadas por modelo ([-,+]): ",pred_s,"->",clase_pred(pred_s))#pred->[-,+]
    print("Crítica: ", text)


Las probabilidades calculadas, las cuales se podrían definir como una forma continua de clasificación parecen entregar más información que el juicio final, puesto que hay veces donde las probabilidades de ambas clases se parecen y finalmente se juzga mal. La clasificación final en cambio solo nos indica si la crítica es positiva o negativa, sin entre medios.

Por otro lado, es dificil que el clasificador (tal como está construido) logre detectar correctamente cuando se está usando una palabra como sarcasmo y cuando no, por lo que críticas negativas que usan palabras como positivas podrían ser clasificadas como positivas.

<img src="https://theonlyblogworthreading.files.wordpress.com/2012/06/science-sarcasm-professor-frink-comic-book-guy-631.jpg" title="sarcasm" width="20%" />

### s) Conlusión de la sección
En esta sección, desarrollamos varios modelos de aprendizaje. El mejor escojido, según exactitud de evaluación. fué el modelo de clasificación SVM con kernel de tipo rbf , C=100 y diccionario filtrado con lematización y usando solo las 4000 palabras más usadas del habla inglesa. La exactitud lograda por este clasificador en el conjunto de prueba fué igual a {{best_model.score(best_features_test,labels_test)}}, o aproximádamente 0.7. 

Al tratarse de algo tan complejo como el lenguaje humano, parece incorrecto señalar que una frase es "positiva" o "negativa" solo viendo sus palabras por separado, por lo que es dificil que se logre una exactitud mayor con un modelo así.

Una propuesta para mejorar el desempeño de estos modelos de clasificación es tomar como variables no solo las frecuencias de las palabras dichas, sino que también la presencia de la interacción entre múltiples palabras. Es decir, dejar de considerar la independencia de cada palabra. Por otro lado, esto aumentaría exponencialmente el número de variantes, lo cual puede significar un problema tanto por el elevado tiempo de ejecución del entrenamiento de los modelos y la llamada maldición de la dimensionalidad.

## Sección 2
Para esta sección se nos solicita obtener el ground truth para un cierto dato a partir de las posibles respuestas que dió gente acerca de este mismo dato. Esto se traduce en obtener el sentimiento correspondiente a una crítica cinematográfica pudiendo ser estos tanto positivos como negativos.

Dentro del dataset se nos entregan diversos valores pero para la resolución de la tarea se utilizaron sólo modificaciones de algunos de estos.

En primera instancia, los datos nos entregan un string correspondiente al comentario acerca de la película, con la ayuda de los vectores pre-entrenados de GloVe, se pudo representar cada palabra del string como un vector, estos ubicados dentro de un espacio vectorial que contrapone palabras opuestas. Ante esta noción, se sumaron todos los vectors correspondientes a un comentario, dando como resultado, una generalización de este. Cuando ya se obtuvo este vector (con diversas dimensiones), se procedió a obtener su norma, entregándonos cómo esta "oración" dista del origen; todo esto con la intención de obtener datos cuyo manejo sea más cómodo.

Ahora bien se tiene un número que nos referencia donde se puede ubicar una oración dentro del plano, se necesitaba de un dato que pueda diferenciar oraciones positivas de negativas, así fue que se obtuvo la columna "resta" que es la sustracción de la cantidad de veces que las personas etiquetaban como positivo a un comentario, menos la cantidad de veces que lo etiquetaban como negativo, así, para un comentario categorizado mayormente como positivo el valor de "resta" toma valores mayores a cero, mientras que para comentarios mayoritariamente negativos se obtiene un valor negativo.

In [ ]:
import pandas as pd
import numpy as np
import re
from numpy import linalg as LA

embeddings_dict = {}
with open("glove.6B.50d.txt", 'r',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

#funcion que le quita todos los caracteres especiales a un string y retorna un numpy array con todas las palabras de esta oracion
def nospace(array):
    out_string = ""
    for wordi in array:
        out_string += re.sub(r'[\W]', '', wordi) + " "#'outstring' es el string que no tiene caracteres especiales, sólo palabras
    values2 = out_string.split() ##a este vector se le quitan los espacios
    words = np.asarray(values2)
    return words

data = pd.read_csv("./mturk-datasets/sentiment_polarity/mturk_answers.csv") 
data.rename(columns={'Input.id':'inputid'}, inplace=True)
data.inputid.unique().shape[0]
a = np.empty(data.inputid.unique().shape[0])
b = np.empty(data.inputid.unique().shape[0])
count = 0
dicc = {}
for i in data.inputid.unique():
    posis = data.loc[data['inputid'] == i].loc[data['Answer.sent']=='pos'].shape[0]#entrega la cantidad de comentarios positivos para un id 
    negs = data.loc[data['inputid'] == i].loc[data['Answer.sent']=='neg'].shape[0]#entrega la cantidad de comentarios positivos para un id
    if posis >= negs:#si hay mas positivos que negativos
        mayoria = 0 # 0 = comentario positivo 
    else:
        mayoria = 1 # 1 = comentario negativo
    if data.loc[data['inputid'] == i]['Input.true_sent'].unique()[0] =='pos':
        realans = 0
    else:
        realans = 1
    array=(data.loc[data['inputid'] == i]['Input.original_sentence'].unique())[0].split()
    print('array es: \n')
    print(array)
    words = nospace(array)
    print('words es: \n')
    print(words)
    vecword = np.zeros((50,), dtype='float32')
    for j in words:
        try:
            vecword = embeddings_dict[j] + vecword
        except KeyError:
            null=np.zeros((50,), dtype='float32')
            vecword = null + vecword
    print('vecword es: \n')
    print(vecword)
    norm_sentence=LA.norm(vecword)
    resta = posis-negs
    dicc[i]=((data.loc[data['inputid'] == i]['Input.original_sentence'].unique())[0],norm_sentence,resta,posis,negs,mayoria,realans)
    a[count]=i
    count=count+1
final_df = pd.DataFrame.from_dict(dicc, orient='index')

In [ ]:
#se agrega el id como una columna en el dataset ya que antes el input_id era el índice
final_df.reset_index(inplace=True)
 
print(final_df)

In [ ]:
#se renombran las columnas del dataset
final_df.columns = ['inputid','osentence','norm_sentence','resta','pos','neg','answer','realans']
print(final_df)

In [ ]:
X=final_df[['norm_sentence','resta']]
Y=final_df['answer']
Y_real=final_df['realans']
#separados ya las X y los Y se procede a crear la división entre los datos para el train y para el test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
import matplotlib.pyplot as plt
fig, axis = plt.subplots(1, 1,figsize=(8, 4))
axis.scatter(x_train['norm_sentence'], x_train['resta'], s=1, c=y_train, cmap=plt.cm.winter)

Al graficar los datos fue posible divisar que correspondía a un problema linealmente separable.

## Regresión logística

Dada la linealidad que presenta el problema, sumado al caso de que este consiste en la asignación de clases a un determinado dato es que es factible la aplicación del modelo de regresión logística.

In [ ]:
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

model = linear_model.LogisticRegression()
model.fit(x_train,y_train)

# Making predictions
y_pred_sk = model.predict(x_test)


# Accuracy
acc_test = accuracy_score(y_test, y_pred_sk)
print("Accuracy para las etiquetas de la gente: "+str(acc_test))
dimy=x_test.shape[0]
y_real=np.zeros((dimy,), dtype='float32')
counting=0
for i in x_test.index:
    y_real[counting]=Y_real[i]
    counting=counting+1
acc_real = accuracy_score(y_pred_sk, y_real)
print("Accuracy para las etiquetas reales: "+str(acc_real))

## SVM lineal

Dada la linealidad del problema también se emplea la máquina "support vector machine" en su modalidad lineal. 
Para su correcta utilización es necesario cambiar la codificación de las etiquetas, transmutando los valores 0 a -1.

In [ ]:
from sklearn import svm
from sklearn.metrics import confusion_matrix

svm_y_train = np.zeros((y_train.shape[0],), dtype='float32')
svm_y_test = np.zeros((y_test.shape[0],), dtype='float32')
svm_y_real = np.zeros((y_real.shape[0],), dtype='float32')
counting = 0
for i in y_train:
    if i == 0:
        svm_y_train[counting]=-1
    else:
        svm_y_train[counting]=1
    counting=counting+1

counting = 0
for i in y_test:
    if i == 0:
        svm_y_test[counting]=-1
    else:
        svm_y_test[counting]=1
    counting=counting+1

counting = 0
for i in y_real:
    if i == 0:
        svm_y_real[counting]=-1
    else:
        svm_y_real[counting]=1
    counting=counting+1   

clf = svm.SVC(kernel='linear', C=1)
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

svm_acc_test = accuracy_score(y_test, y_pred)
svm_acc_real = accuracy_score(y_real, y_pred)

print("Accuracy para las etiquetas de la gente: "+str(svm_acc_test))
print("Accuracy para las etiquetas de la gente: "+str(svm_acc_real))
print("Matriz de Confusión: ")
print(confusion_matrix(y_test,y_pred))


Los resultados obtenidos son bastante buenos dada la incertidumbre a la que nos enfrentamos cuando se obtienen muchas respuestas de fuentes no oficiales las cuales yacen sobre la subjetividad de la gente.

Un paso fundamental fue la representación que se le dió a los datos haciendo muchísimo más fácil la tarea de reconocer la verdadera naturaleza de los comentarios.

<a id="refs"></a>
## Referencias
[1] Keras: Deep Learning library for Theano and TensorFlow. https://keras.io/  
[2] https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews  
[3] https://en.wikipedia.org/wiki/Stopwords  
[4] https://en.wikipedia.org/wiki/Lemmatisation  
[5] Landauer, T. K., Foltz, P. W., & Laham, D. (1998). *An introduction to latent semantic analysis*. Discourse processes, 25(2-3), 259-284.  
[6] https://github.com/cjhutto/vaderSentiment  
[7] https://en.wikipedia.org/wiki/Stemming  
[8] https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html  
[9] https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html  
[10] https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html#sklearn.metrics.precision_recall_fscore_support